In [2]:
import re
from tqdm import tqdm
from transformers import AutoTokenizer
import os
import sys
import pickle

# root_dir = os.path.abspath(os.path.dirname(os.path.dirname(__file__)))
# sys.path.append(root_dir)

DEFAULT_MODEL_NAME = 'roberta-base'

def clean(data):
    tmp_doc = []
    for words in data.split():
        if ':' not in words and '@' not in words and len(words) <= 60:
            c = re.sub(r'[>|-]', '', words)
            if len(c) > 0:
                tmp_doc.append(c)
    tmp_doc = ' '.join(tmp_doc)
    tmp_doc = re.sub(r'\([A-Za-z \.]*[A-Z][A-Za-z \.]*\) ', '', tmp_doc)
    return tmp_doc

s = "The Pacific Economic Cooperation Council (PECC) has issued a report examining the economic impact of the COVID-19 crisis on the Asia-Pacific region and outlining an agenda for cooperation.The technical and policy-focused publication titled, ‘State of the Region: Special report on COVID-19,’ acknowledges high levels of policy uncertainty in the region due to the pandemic, as well as the scale and duration of related economic and social shocks, and pace of recovery. The report thus focuses on how regional cooperation can provide governments with more options for recovery in the face of these uncertainties.The report collates a set of proposals using data collected from 710 survey respondents in business, academia, government, and civil society between 9 May and 12 June 2020. The results show greater levels of pessimism on economic impacts to the region than official estimates indicate. The report points to regional stimulus efforts totaling approximately USD 5.4 trillion, and notes that while policymakers’ appetites are constrained by memories of recent financial crises, “regional economies have space for further stimulus.”Regional mechanisms, the authors emphasize, can facilitate the design and implementation of coordination and cooperation packages, and build a sense of direction to support future growth. They note that top priorities for regional cooperation include sharing pandemic preparedness practices, vaccine development, and three aspects of trade with respect to essential products: the facilitation of trade as a whole; the removal of export restrictions; and the removal of tariffs.Observing that the pandemic has both deepened and accelerated preexisting trends, the report notes the importance of human contact, but also opportunities around digital technology and the multitude of connections available. As schools remain shuttered in many parts of the world, the report notes remote learning opportunities, with the caveat that risks remain around the digital divide despite action being taken through the Asia-Pacific Economic Cooperation (APEC) Internet and Digital Roadmap.To address the “first order” priorities stemming from the pandemic, the report recommends multilateral actions that facilitate regional progress on seven issue areas:Information sharing;Flow of essential products;Moving beyond gross domestic product (GDP);Facilitating e-commerce;Restarting travel;Minimizing disruption to supply chains; andContact tracing.A message from PECC co-chairs Don Campbell and Su Ge highlights that this year marks PECC’s 40th anniversary and that the rationale behind the Council’s establishment still rings true today. They note that “[w]e can only resolve this pandemic and economic crisis through effective cooperation.” [Publication:State of the Region Report: Impact of the Covid‐19 Crisis] [Publication Landing Page]"

/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

2780

In [31]:
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL_NAME)

def clean(text):
    text = re.sub(r'\s+', ' ', text)
    return text

tokenizer.decode(tokenizer.encode(clean(s)))

Token indices sequence length is longer than the specified maximum sequence length for this model (557 > 512). Running this sequence through the model will result in indexing errors


'<s>The Pacific Economic Cooperation Council (PECC) has issued a report examining the economic impact of the COVID-19 crisis on the Asia-Pacific region and outlining an agenda for cooperation.The technical and policy-focused publication titled, ‘State of the Region: Special report on COVID-19,’ acknowledges high levels of policy uncertainty in the region due to the pandemic, as well as the scale and duration of related economic and social shocks, and pace of recovery. The report thus focuses on how regional cooperation can provide governments with more options for recovery in the face of these uncertainties.The report collates a set of proposals using data collected from 710 survey respondents in business, academia, government, and civil society between 9 May and 12 June 2020. The results show greater levels of pessimism on economic impacts to the region than official estimates indicate. The report points to regional stimulus efforts totaling approximately USD 5.4 trillion, and notes t

In [23]:
import stanza
nlp = stanza.Pipeline(lang='en', processors='tokenize', verbose=False)

class Block:
    tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL_NAME)
    def __init__(self, ids, pos, blk_type=1):
        self.ids = ids
        self.pos = pos
        self.relevance = 0
        self.estimation = 0
    
    def add_sentence_ids(self, ids):
        self.ids.extend(ids)
        
    def ß__lt__(self, rhs):
        return self.blk_type < rhs.blk_type or (self.blk_type == rhs.blk_type and self.pos < rhs.pos)
    
    def __ne__(self, rhs):
        return self.pos != rhs.pos or self.blk_type != rhs.blk_type
    
    def __len__(self):
        return len(self.ids)
    
    def __str__(self):
        return Block.tokenizer.convert_tokens_to_string(Block.tokenizer.convert_ids_to_tokens(self.ids))


# split text into sentences
def split_text_into_sentences(text):
    text = re.sub(r'\s+', ' ', text)
    doc = nlp(text)
    sentences = [sentence.text for sentence in doc.sentences]
    return sentences

# group a list of sentences into blocks of size BLOCK_SIZE
def group_sentences_into_blocks(sentences, block_size=63):
    tokenzier = AutoTokenizer.from_pretrained(DEFAULT_MODEL_NAME)
    
    blocks = []
    current_position = 0
    current_block = Block([], current_position)
    for sentence in sentences:
        tokenized_sentence = tokenzier.encode(sentence)
        
        if len(current_block) + len(tokenized_sentence) <= block_size:
            current_block.add_sentence_ids(tokenized_sentence)
        else:
            blocks.append(current_block)
            current_position += 1
            
            if len(tokenized_sentence) > block_size:
                tokenized_sentence = tokenized_sentence[:block_size]
                print(f"Truncation at position {current_position} with sentence '{sentence}'")
            current_block = Block(tokenized_sentence, current_position)
    blocks.append(current_block)
    
    return blocks
        
        

sentences = split_text_into_sentences(s)
blocks = group_sentences_into_blocks(sentences)

[0, 133, 3073, 4713, 18204, 1080, 36, 510, 3586, 347, 43, 34, 1167, 10, 266, 14951, 5, 776, 913, 9, 5, 6247, 43814, 12, 1646, 1486, 15, 5, 1817, 12, 8145, 976, 8, 24387, 41, 4026, 13, 4601, 4, 2]
<s>The Pacific Economic Cooperation Council (PECC) has issued a report examining the economic impact of the COVID-19 crisis on the Asia-Pacific region and outlining an agenda for cooperation.</s>
[0, 133, 3165, 8, 714, 12, 12804, 5362, 6593, 6, 44, 711, 13360, 9, 5, 6131, 35, 3672, 266, 15, 6247, 43814, 12, 1646, 6, 17, 27, 17748, 239, 1389, 9, 714, 4983, 11, 5, 976, 528, 7, 5, 23387, 14414, 6, 25, 157, 25, 5, 3189, 8, 13428, 9, 1330, 776, 8, 592, 23485, 6, 8, 2877, 9, 2752, 4, 2]
<s>The technical and policy-focused publication titled, ‘State of the Region: Special report on COVID-19,’ acknowledges high levels of policy uncertainty in the region due to the pandemic, as well as the scale and duration of related economic and social shocks, and pace of recovery.</s>
[0, 133, 266, 4634, 7235, 15, 

In [20]:
from nltk.tokenize import sent_tokenize

def split_text_into_sentences(text):
    text = re.sub(r'\s+', ' ', text)
    doc = sent_tokenize(text)
    return doc

split_text_into_sentences(s)

['The Pacific Economic Cooperation Council (PECC) has issued a report examining the economic impact of the COVID-19 crisis on the Asia-Pacific region and outlining an agenda for cooperation.The technical and policy-focused publication titled, ‘State of the Region: Special report on COVID-19,’ acknowledges high levels of policy uncertainty in the region due to the pandemic, as well as the scale and duration of related economic and social shocks, and pace of recovery.',
 'The report thus focuses on how regional cooperation can provide governments with more options for recovery in the face of these uncertainties.The report collates a set of proposals using data collected from 710 survey respondents in business, academia, government, and civil society between 9 May and 12 June 2020.',
 'The results show greater levels of pessimism on economic impacts to the region than official estimates indicate.',
 'The report points to regional stimulus efforts totaling approximately USD 5.4 trillion, a

In [27]:
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL_NAME)
tokenizer.decode(tokenizer.encode(s))

Token indices sequence length is longer than the specified maximum sequence length for this model (562 > 512). Running this sequence through the model will result in indexing errors


'<s>The Pacific Economic Cooperation Council (PECC) has issued a report examining the economic impact of the COVID-19 crisis on the Asia-Pacific region and outlining an agenda for cooperation.The technical and policy-focused publication titled, ‘State of the Region: Special report on COVID-19,’ acknowledges\xa0high levels of policy uncertainty in the region due to the pandemic,\xa0as well as the scale and duration of related economic and social shocks, and pace of recovery. The report thus focuses on how regional cooperation can provide governments with more options for recovery in the face of these uncertainties.The report collates a set of proposals using data collected from 710 survey respondents in business, academia, government, and civil society between 9 May and 12 June 2020. The results show greater levels of pessimism on economic impacts to the region than official estimates indicate. The report points to regional stimulus efforts totaling approximately USD 5.4 trillion, and n

In [28]:
list(range(0,10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]